In [1]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.6 MB/s eta 0:00:00


In [2]:
!wget https://ftp.ncbi.nlm.nih.gov/blast/db/FASTA/swissprot.gz

--2023-03-19 14:02:02--  https://ftp.ncbi.nlm.nih.gov/blast/db/FASTA/swissprot.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 165.112.9.228, 2607:f220:41f:250::230, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 142310285 (136M) [application/x-gzip]
Saving to: ‘swissprot.gz’

swissprot.gz        100%[===================>] 135.72M  16.8MB/s    in 9.4s    

2023-03-19 14:02:12 (14.4 MB/s) - ‘swissprot.gz’ saved [142310285/142310285]



Ejecute la siguiente línea para descomprimir el archivo de base de datos comprimido.

In [3]:
!gunzip swissprot.gz

Ahora que hemos probado la versión basada en Internet, vamos a hacer un BLASTing local en la base de datos Swiss-Prot que hemos descargado. Primero ejecuta la celda de abajo para definir nuestra matriz Blosum62 que será usada para nuestros alineamientos.

In [4]:
blosum62 = {
    ('W', 'F'): 1, ('L', 'R'): -2, ('S', 'P'): -1, ('V', 'T'): 0,
    ('Q', 'Q'): 5, ('N', 'A'): -2, ('Z', 'Y'): -2, ('W', 'R'): -3,
    ('Q', 'A'): -1, ('S', 'D'): 0, ('H', 'H'): 8, ('S', 'H'): -1,
    ('H', 'D'): -1, ('L', 'N'): -3, ('W', 'A'): -3, ('Y', 'M'): -1,
    ('G', 'R'): -2, ('Y', 'I'): -1, ('Y', 'E'): -2, ('B', 'Y'): -3,
    ('Y', 'A'): -2, ('V', 'D'): -3, ('B', 'S'): 0, ('Y', 'Y'): 7,
    ('G', 'N'): 0, ('E', 'C'): -4, ('Y', 'Q'): -1, ('Z', 'Z'): 4,
    ('V', 'A'): 0, ('C', 'C'): 9, ('M', 'R'): -1, ('V', 'E'): -2,
    ('T', 'N'): 0, ('P', 'P'): 7, ('V', 'I'): 3, ('V', 'S'): -2,
    ('Z', 'P'): -1, ('V', 'M'): 1, ('T', 'F'): -2, ('V', 'Q'): -2,
    ('K', 'K'): 5, ('P', 'D'): -1, ('I', 'H'): -3, ('I', 'D'): -3,
    ('T', 'R'): -1, ('P', 'L'): -3, ('K', 'G'): -2, ('M', 'N'): -2,
    ('P', 'H'): -2, ('F', 'Q'): -3, ('Z', 'G'): -2, ('X', 'L'): -1,
    ('T', 'M'): -1, ('Z', 'C'): -3, ('X', 'H'): -1, ('D', 'R'): -2,
    ('B', 'W'): -4, ('X', 'D'): -1, ('Z', 'K'): 1, ('F', 'A'): -2,
    ('Z', 'W'): -3, ('F', 'E'): -3, ('D', 'N'): 1, ('B', 'K'): 0,
    ('X', 'X'): -1, ('F', 'I'): 0, ('B', 'G'): -1, ('X', 'T'): 0,
    ('F', 'M'): 0, ('B', 'C'): -3, ('Z', 'I'): -3, ('Z', 'V'): -2,
    ('S', 'S'): 4, ('L', 'Q'): -2, ('W', 'E'): -3, ('Q', 'R'): 1,
    ('N', 'N'): 6, ('W', 'M'): -1, ('Q', 'C'): -3, ('W', 'I'): -3,
    ('S', 'C'): -1, ('L', 'A'): -1, ('S', 'G'): 0, ('L', 'E'): -3,
    ('W', 'Q'): -2, ('H', 'G'): -2, ('S', 'K'): 0, ('Q', 'N'): 0,
    ('N', 'R'): 0, ('H', 'C'): -3, ('Y', 'N'): -2, ('G', 'Q'): -2,
    ('Y', 'F'): 3, ('C', 'A'): 0, ('V', 'L'): 1, ('G', 'E'): -2,
    ('G', 'A'): 0, ('K', 'R'): 2, ('E', 'D'): 2, ('Y', 'R'): -2,
    ('M', 'Q'): 0, ('T', 'I'): -1, ('C', 'D'): -3, ('V', 'F'): -1,
    ('T', 'A'): 0, ('T', 'P'): -1, ('B', 'P'): -2, ('T', 'E'): -1,
    ('V', 'N'): -3, ('P', 'G'): -2, ('M', 'A'): -1, ('K', 'H'): -1,
    ('V', 'R'): -3, ('P', 'C'): -3, ('M', 'E'): -2, ('K', 'L'): -2,
    ('V', 'V'): 4, ('M', 'I'): 1, ('T', 'Q'): -1, ('I', 'G'): -4,
    ('P', 'K'): -1, ('M', 'M'): 5, ('K', 'D'): -1, ('I', 'C'): -1,
    ('Z', 'D'): 1, ('F', 'R'): -3, ('X', 'K'): -1, ('Q', 'D'): 0,
    ('X', 'G'): -1, ('Z', 'L'): -3, ('X', 'C'): -2, ('Z', 'H'): 0,
    ('B', 'L'): -4, ('B', 'H'): 0, ('F', 'F'): 6, ('X', 'W'): -2,
    ('B', 'D'): 4, ('D', 'A'): -2, ('S', 'L'): -2, ('X', 'S'): 0,
    ('F', 'N'): -3, ('S', 'R'): -1, ('W', 'D'): -4, ('V', 'Y'): -1,
    ('W', 'L'): -2, ('H', 'R'): 0, ('W', 'H'): -2, ('H', 'N'): 1,
    ('W', 'T'): -2, ('T', 'T'): 5, ('S', 'F'): -2, ('W', 'P'): -4,
    ('L', 'D'): -4, ('B', 'I'): -3, ('L', 'H'): -3, ('S', 'N'): 1,
    ('B', 'T'): -1, ('L', 'L'): 4, ('Y', 'K'): -2, ('E', 'Q'): 2,
    ('Y', 'G'): -3, ('Z', 'S'): 0, ('Y', 'C'): -2, ('G', 'D'): -1,
    ('B', 'V'): -3, ('E', 'A'): -1, ('Y', 'W'): 2, ('E', 'E'): 5,
    ('Y', 'S'): -2, ('C', 'N'): -3, ('V', 'C'): -1, ('T', 'H'): -2,
    ('P', 'R'): -2, ('V', 'G'): -3, ('T', 'L'): -1, ('V', 'K'): -2,
    ('K', 'Q'): 1, ('R', 'A'): -1, ('I', 'R'): -3, ('T', 'D'): -1,
    ('P', 'F'): -4, ('I', 'N'): -3, ('K', 'I'): -3, ('M', 'D'): -3,
    ('V', 'W'): -3, ('W', 'W'): 11, ('M', 'H'): -2, ('P', 'N'): -2,
    ('K', 'A'): -1, ('M', 'L'): 2, ('K', 'E'): 1, ('Z', 'E'): 4,
    ('X', 'N'): -1, ('Z', 'A'): -1, ('Z', 'M'): -1, ('X', 'F'): -1,
    ('K', 'C'): -3, ('B', 'Q'): 0, ('X', 'B'): -1, ('B', 'M'): -3,
    ('F', 'C'): -2, ('Z', 'Q'): 3, ('X', 'Z'): -1, ('F', 'G'): -3,
    ('B', 'E'): 1, ('X', 'V'): -1, ('F', 'K'): -3, ('B', 'A'): -2,
    ('X', 'R'): -1, ('D', 'D'): 6, ('W', 'G'): -2, ('Z', 'F'): -3,
    ('S', 'Q'): 0, ('W', 'C'): -2, ('W', 'K'): -3, ('H', 'Q'): 0,
    ('L', 'C'): -1, ('W', 'N'): -4, ('S', 'A'): 1, ('L', 'G'): -4,
    ('W', 'S'): -3, ('S', 'E'): 0, ('H', 'E'): 0, ('S', 'I'): -2,
    ('H', 'A'): -2, ('S', 'M'): -1, ('Y', 'L'): -1, ('Y', 'H'): 2,
    ('Y', 'D'): -3, ('E', 'R'): 0, ('X', 'P'): -2, ('G', 'G'): 6,
    ('G', 'C'): -3, ('E', 'N'): 0, ('Y', 'T'): -2, ('Y', 'P'): -3,
    ('T', 'K'): -1, ('A', 'A'): 4, ('P', 'Q'): -1, ('T', 'C'): -1,
    ('V', 'H'): -3, ('T', 'G'): -2, ('I', 'Q'): -3, ('Z', 'T'): -1,
    ('C', 'R'): -3, ('V', 'P'): -2, ('P', 'E'): -1, ('M', 'C'): -1,
    ('K', 'N'): 0, ('I', 'I'): 4, ('P', 'A'): -1, ('M', 'G'): -3,
    ('T', 'S'): 1, ('I', 'E'): -3, ('P', 'M'): -2, ('M', 'K'): -1,
    ('I', 'A'): -1, ('P', 'I'): -3, ('R', 'R'): 5, ('X', 'M'): -1,
    ('L', 'I'): 2, ('X', 'I'): -1, ('Z', 'B'): 1, ('X', 'E'): -1,
    ('Z', 'N'): 0, ('X', 'A'): 0, ('B', 'R'): -1, ('B', 'N'): 3,
    ('F', 'D'): -3, ('X', 'Y'): -1, ('Z', 'R'): 0, ('F', 'H'): -1,
    ('B', 'F'): -3, ('F', 'L'): 0, ('X', 'Q'): -1, ('B', 'B'): 4
}
Blosum62 = {}
for i in blosum62:
  Blosum62[i] = blosum62[i]
  Blosum62[i[::-1]] = blosum62[i] #adds the reverse pairs so that we can lookup ('F', 'H') or ('H', 'F') for example

A continuación intentaremos BLASTing para la proteína S-ribosilhomocisteína liasa. "mpsvesfeld hnavvapyvr hcgvhkvgtd gvvnkfdirf cqpnkqamkp dtihtlehllaftirshaek ydhfdiidis pmgcqtgyyl vvsgeptsaeivdlledtmk eaveiteipaanekqcgqak lhdlegakrl mrfwlsqdke ellkvfg". La celda siguiente ejecutará primero una función miniblast.read_database_fasta() que accederá a las líneas de la base de datos Swiss-Prot que residen en el servidor. A continuación, la función miniblast.best_alignment_fasta() ejecutará la búsqueda, con una secuencia de consulta, la base de datos y el tamaño de palabra o k-mer como parámetros de entrada. La salida resultante se adjuntará a una variable que llamaremos resultado, y en un bucle posterior podremos imprimir el contenido del resultado.

In [7]:
import sys
from os import path

if path.exists("/content/bioinformatics_stockholm/"):
  pass
else:
  !git clone https://github.com/majomonte/Proyecto-Final-WBDs-La-Camp.git
sys.path.insert(0,'/content')

import importlib.util
spec = importlib.util.spec_from_file_location("msa", "/content/bioinformatics_stockholm/msa/msa.py")
msa = importlib.util.module_from_spec(spec)
spec.loader.exec_module(msa)

spec = importlib.util.spec_from_file_location("miniblast", "/content/bioinformatics_stockholm/Miniblast/miniblast.py")
miniblast = importlib.util.module_from_spec(spec)
spec.loader.exec_module(miniblast)

In [6]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import pairwise2
import numpy as np
db = miniblast.read_database_fasta('/content/swissprot')
query = "mpsvesfeldhnavvapyvrhcgvhkvgtdgvvnkfdirfcqpnkqamkpdtihtlehllaftirshaekydhfdiidispmgcqtgyylvvsgeptsaeivdlledtmkeaveiteipaanekqcgqaklhdlegakrlmrfwlsqdkeellkvfg"
result = miniblast.best_alignment_fasta(db,query,w=3)
print(result)

()


In [8]:
for i in result:
  print(i)
  print(db[i])

Obsérvese cómo se obtienen algunas coincidencias adicionales aparentemente no relacionadas con nuestra consulta (que ya sabemos que es la albúmina). Para entender mejor qué coincidencias son auténticas, siempre es importante examinar las alineaciones directamente. Este módulo miniblast no realiza el alineamiento local final Smith-Waterman típico en el flujo de trabajo BLAST, por lo que podemos volver a hacer uso de la función pairwise2.align() de biopython para realizar el alineamiento local final en los resultados ganadores con nuestra consulta. Simplemente estableceremos un bucle que recorra la base de datos, buscando los índices producidos por la función BLAST anterior, y que realice el alineamiento con nuestra consulta y formatee para ver visualmente los alineamientos como hicimos con los datos NGS la última vez.

In [9]:
for i in result:
  print(db[i])
  target_sequence = db[i].seq
  aln = pairwise2.align.localds(query,target_sequence, match_dict = Blosum62, open = -3, extend = -1)[0]
  print(pairwise2.format_alignment(align1=aln[0], align2=aln[1], score=aln[2], begin=aln[3], end=aln[4], full_sequences=True))